In [1]:
"""
残差连接（Residual Connection）

残差连接是ResNet的核心创新，解决了深度神经网络的退化问题。

核心思想：
y = F(x) + x

其中F(x)是残差映射，通过学习残差而非直接学习目标映射，
使得网络更容易优化。当需要学习恒等映射时，只需将F(x)的权重趋向于0即可。

优势：
1. 缓解梯度消失问题：梯度可以通过短路连接直接传播
2. 加速训练收敛：提供了梯度的"高速公路"
3. 允许构建更深的网络：ResNet-152等超深网络
4. 提升模型性能：在图像分类、目标检测等任务中表现优异

数学原理：
反向传播时，梯度可以直接通过恒等映射传递：
∂L/∂x = ∂L/∂y * (∂F(x)/∂x + 1)
即使∂F(x)/∂x很小，+1项保证了梯度不会消失
"""

from keras import layers
from keras.models import Model
from keras.layers import Input
import numpy as np

2025-12-12 22:20:48.971015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 22:20:48.977290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-12 22:20:48.985081: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-12 22:20:48.987328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-12 22:20:48.993500: I tensorflow/core/platform/cpu_feature_guar

In [2]:
"""
场景1：特征图尺寸和通道数相同时的残差连接

当输入和输出的shape完全一致时，可以直接相加
"""

# 定义输入：(batch_size, height, width, channels)
input_tensor = Input(shape=(32, 32, 256))

# 残差块：两个3x3卷积
x = input_tensor
y = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
y = layers.Conv2D(256, 3, padding='same', activation='relu')(y)

# 残差连接：将输入x直接加到输出y上
output_tensor = layers.add([x, y])

# 构建模型
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

# 验证可运行性
test_input = np.random.randn(1, 32, 32, 256).astype('float32')
output = model.predict(test_input, verbose=0)
print(f"输入shape: {test_input.shape}")
print(f"输出shape: {output.shape}")
print("残差连接验证成功：输入输出shape一致")

I0000 00:00:1765549250.101218   33744 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549250.123259   33744 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549250.124370   33744 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549250.126592   33744 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32,    │          0 │ -                 │
│ (InputLayer)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │    590,080 │ input_layer[0][0] │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d[0][0]      │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ input_layer[0][0… │
│                     │ 256)              │            │ conv2d_1[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,180,160 (4.50 MB)

 Trainable params: 1,180,160 (4.50 MB)

 Non-trainable params: 0 (0.00 B)

I0000 00:00:1765549250.695597   33891 service.cc:146] XLA service 0x767c480037b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765549250.695613   33891 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9
2025-12-12 22:20:50.698740: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-12 22:20:50.707148: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 91301


输入shape: (1, 32, 32, 256)
输出shape: (1, 32, 32, 256)
残差连接验证成功：输入输出shape一致


I0000 00:00:1765549250.961351   33891 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [3]:
"""
场景2：特征图尺寸或通道数不同时的残差连接

当经过步长>1的卷积或池化后，特征图尺寸会改变，此时需要使用1x1卷积
对shortcut分支进行调整，使其与主分支的shape匹配

常用调整策略：
1. 1x1卷积 + 步长调整：改变通道数和空间尺寸
2. 投影快捷连接（Projection Shortcut）：ResNet论文中的方法
"""

# 定义输入
input_tensor = Input(shape=(32, 32, 128))

x = input_tensor

# 主分支：包含下采样操作
y = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
y = layers.Conv2D(256, 3, padding='same', activation='relu')(y)
y = layers.MaxPooling2D(2, strides=2)(y)  # 空间尺寸减半

# shortcut分支：使用1x1卷积匹配主分支的shape
# 同时调整通道数(128->256)和空间尺寸(stride=2)
shortcut = layers.Conv2D(256, 1, strides=2, padding='same')(x)

# 残差连接
output_tensor = layers.add([shortcut, y])

# 构建模型
model_downsample = Model(inputs=input_tensor, outputs=output_tensor)
model_downsample.summary()

# 验证可运行性
test_input = np.random.randn(1, 32, 32, 128).astype('float32')
output = model_downsample.predict(test_input, verbose=0)
print(f"\n输入shape: {test_input.shape}")
print(f"输出shape: {output.shape}")
print("下采样残差连接验证成功：通道数从128增至256，空间尺寸减半")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32,    │          0 │ -                 │
│ (InputLayer)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │    295,168 │ input_layer_1[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_2[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 16, 16,    │     33,024 │ input_layer_1[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 16, 16,    │          0 │ conv2d_4[0][0],   │
│                     │ 256)              │            │ max_pooling2d[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 918,272 (3.50 MB)

 Trainable params: 918,272 (3.50 MB)

 Non-trainable params: 0 (0.00 B)


输入shape: (1, 32, 32, 128)
输出shape: (1, 16, 16, 256)
下采样残差连接验证成功：通道数从128增至256，空间尺寸减半
